In [1]:
from dj_notebook import activate

plus = activate()

The .env file has been loaded. See base.py for more information

Output()

/Users/jochen/.virtualenvs/pp/lib/python3.12/site-packages/django/db/backends/utils.py:98: RuntimeWarning: 
Accessing the database during app initialization is discouraged. To fix this warning, avoid executing queries in 
AppConfig.ready() or when your app modules are imported.
  warnings.warn(self.APPS_NOT_READY_WARNING_MSG, category=RuntimeWarning)

In [2]:
import sqlparse


def format_sql(query):
    return sqlparse.format(query, reindent=True, keyword_case='upper')

def get_formatted_sql(queryset):
    # Get the raw SQL query from the queryset
    sql, params = queryset.query.sql_with_params()
    
    # Format the SQL query
    formatted_sql = format_sql(sql % params)
    
    return formatted_sql

## Minimal Example

In [3]:
queryset = Page.objects.all()
queryset = queryset.filter(slug="1\x00����%2527%2522")
# print(queryset.count())

NameError: name 'Page' is not defined

In [7]:
from cast.models import Post
from cast.filters import PostFilterset

queryset = Post.objects.all()
filterset = PostFilterset({"slug": "1\x00����%2527%2522"}, queryset)
print(filterset.form.is_valid())
print(dict(filterset.form.errors))
print(get_formatted_sql(filterset.qs))
print(filterset.qs.count())

False
{'slug': ['Nullzeichen sind nicht erlaubt.']}
SELECT "wagtailcore_page"."id",
       "wagtailcore_page"."path",
       "wagtailcore_page"."depth",
       "wagtailcore_page"."numchild",
       "wagtailcore_page"."translation_key",
       "wagtailcore_page"."locale_id",
       "wagtailcore_page"."latest_revision_id",
       "wagtailcore_page"."live",
       "wagtailcore_page"."has_unpublished_changes",
       "wagtailcore_page"."first_published_at",
       "wagtailcore_page"."last_published_at",
       "wagtailcore_page"."live_revision_id",
       "wagtailcore_page"."go_live_at",
       "wagtailcore_page"."expire_at",
       "wagtailcore_page"."expired",
       "wagtailcore_page"."locked",
       "wagtailcore_page"."locked_at",
       "wagtailcore_page"."locked_by_id",
       "wagtailcore_page"."title",
       "wagtailcore_page"."draft_title",
       "wagtailcore_page"."slug",
       "wagtailcore_page"."content_type_id",
       "wagtailcore_page"."url_path",
       "wagtailcore_pag

In [5]:
from django.conf import settings
from django.test import RequestFactory

from wagtail.api.v2.views import PagesAPIViewSet


settings.ALLOWED_HOSTS.append("testserver")
rf = RequestFactory()
input_with_null_byte = "1%00%EF%BF%BD%EF%BF%BD%EF%BF%BD%EF%BF%BD%252527%252522"
request = rf.get(f"?slug={input_with_null_byte}")
viewset = PagesAPIViewSet()
viewset.request = request
queryset = viewset.get_queryset()
queryset = viewset.filter_queryset(queryset)
print(get_formatted_sql(queryset))
print(queryset.count())

SELECT "wagtailcore_page"."id",
       "wagtailcore_page"."path",
       "wagtailcore_page"."depth",
       "wagtailcore_page"."numchild",
       "wagtailcore_page"."translation_key",
       "wagtailcore_page"."locale_id",
       "wagtailcore_page"."latest_revision_id",
       "wagtailcore_page"."live",
       "wagtailcore_page"."has_unpublished_changes",
       "wagtailcore_page"."first_published_at",
       "wagtailcore_page"."last_published_at",
       "wagtailcore_page"."live_revision_id",
       "wagtailcore_page"."go_live_at",
       "wagtailcore_page"."expire_at",
       "wagtailcore_page"."expired",
       "wagtailcore_page"."locked",
       "wagtailcore_page"."locked_at",
       "wagtailcore_page"."locked_by_id",
       "wagtailcore_page"."title",
       "wagtailcore_page"."draft_title",
       "wagtailcore_page"."slug",
       "wagtailcore_page"."content_type_id",
       "wagtailcore_page"."url_path",
       "wagtailcore_page"."owner_id",
       "wagtailcore_page"."seo_title"

DataError: PostgreSQL text fields cannot contain NUL (0x00) bytes

In [6]:
from cast.models import Post
from cast.filters import PostFilterset

queryset = Post.objects.all()
filterset = PostFilterset({"slug": "design-patterns-draft"}, queryset)
print(get_formatted_sql(filterset.qs))
print(filterset.qs.count())

SELECT "wagtailcore_page"."id",
       "wagtailcore_page"."path",
       "wagtailcore_page"."depth",
       "wagtailcore_page"."numchild",
       "wagtailcore_page"."translation_key",
       "wagtailcore_page"."locale_id",
       "wagtailcore_page"."latest_revision_id",
       "wagtailcore_page"."live",
       "wagtailcore_page"."has_unpublished_changes",
       "wagtailcore_page"."first_published_at",
       "wagtailcore_page"."last_published_at",
       "wagtailcore_page"."live_revision_id",
       "wagtailcore_page"."go_live_at",
       "wagtailcore_page"."expire_at",
       "wagtailcore_page"."expired",
       "wagtailcore_page"."locked",
       "wagtailcore_page"."locked_at",
       "wagtailcore_page"."locked_by_id",
       "wagtailcore_page"."title",
       "wagtailcore_page"."draft_title",
       "wagtailcore_page"."slug",
       "wagtailcore_page"."content_type_id",
       "wagtailcore_page"."url_path",
       "wagtailcore_page"."owner_id",
       "wagtailcore_page"."seo_title"

In [9]:
filterset.form.is_valid()

True

In [10]:
filterset.form.cleaned_data

{'slug': 'design-patterns-draft',
 'search': '',
 'date': None,
 'date_facets': '',
 'o': None}